# The dojo

## Setup training device

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [7]:
import torch
from torch import device, nn
import torchvision
import torchvision.io as io


print(f'PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using device: {device}')

ok = io.read_image("data/E45Vejle_1011.jpg")

PyTorch version: 1.13.1+cu117
torchvision version: 0.13.1
Using device: cpu


AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

## Data loading

### Test dataset

In [3]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

MNIST_train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None,
)

MNIST_test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

BATCH_SIZE = 32

MNIST_class_names = MNIST_train_data.classes

MNIST_train_dataloader = DataLoader(MNIST_train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              drop_last=True) # Drop last

MNIST_test_dataloader = DataLoader(MNIST_test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

print(f'Len of train dataloader: {len(MNIST_train_dataloader)} batches of {BATCH_SIZE}')
print(f'Len of test dataloader: {len(MNIST_test_dataloader)} batches of {BATCH_SIZE}')

Len of train dataloader: 1875 batches of 32
Len of test dataloader: 313 batches of 32


##### Check out what's inside the training dataloader

In [8]:
train_features_batch = next(iter(MNIST_train_dataloader))
MNIST_train_features_batch, MNIST_train_labels_batch = next(iter(MNIST_train_dataloader))

print(F"Train features shape: {MNIST_train_features_batch.shape}")
print(F"Train labels shape: {MNIST_train_labels_batch.shape}")

Train features shape: torch.Size([32, 1, 28, 28])
Train labels shape: torch.Size([32])


### Custom dataset

In [5]:
from torch.utils.data import DataLoader
from utils.dataset import CustomImageDataset

train_data = CustomImageDataset("indices/trainIndex/bremen.csv","data/leftImg8bit/train/bremen/")
test_data = CustomImageDataset("indices/testIndex/berlin.csv","data/leftImg8bit/test/berlin/")

BATCH_SIZE = 32

train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

print(f'Len of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}')
print(f'Len of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}')

Len of train dataloader: 10 batches of 32
Len of test dataloader: 17 batches of 32


##### Check out what's inside the training dataloader

In [6]:
train_features_batch = next(iter(train_dataloader))
train_features_batch, train_labels_batch = next(iter(train_dataloader))

print(F"Train features shape: {train_features_batch.shape}")
print(F"Train labels shape: {train_labels_batch.shape}")

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

## Model

### Create a flatten layer - Testing Flatten()

In [ ]:
flatten_model = nn.Flatten()

x = train_features_batch[0]
print(f'Shape before flattening: {x.shape}')     # torch.Size([3, 1024, 2048])

output = flatten_model(x)
print(f'Shape after flattening: {output.shape}') # torch.Size([3, 2097152])

### Instantiate model

#### Baseline models

In [ ]:
# For testing - do not change
from model.chrome_vision import ChromeVisionModel

test_CPU_baseline_model = ChromeVisionModel(
    input_shape=784,    
    hidden_units=10,    # Units in the hidden layer
    output_shape=len(MNIST_class_names) # one for every class (Arbitrary right now)
).to(device)

model = test_CPU_baseline_model

In [ ]:
# For testing - do not change
from model.chrome_vision import ChromeVisionModelV2

test_GPU_CNN_model = ChromeVisionModelV2(
    input_shape=1,      # Color channel: black/white 
    hidden_units=10,    # Units in the hidden layer
    output_shape=(len(MNIST_class_names)) # Each class
).to(device)

model = test_GPU_CNN_model

In [ ]:
from model.chrome_vision import ChromeVisionModel

model = ChromeVisionModel(
    input_shape= 1024 * 2048,    
    hidden_units=5,    # Units in the hidden layer
    output_shape= 3 # one for every class (Arbitrary right now)
).to(device)

#### MoCo

In [ ]:
from model.chrome_vision import ChromeCut
from model.encoder import ResNet50

encoder = ResNet50(num_classes=len(MNIST_class_names),
                   in_features=1000)

model = ChromeCut(base_encoder=encoder,
                  feature_dim=len(MNIST_class_names),
                  queue_size=5000,
                  momentum=0.9,
                  softmax_temp=0.07,
                  mlp=True).to(device)

### Setup loss function and optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### Training loop - MoCo

In [ ]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
from model.evaluation import train_step, test_step # use torchmetrics.utilities.data.select_topk
from model.utilis import print_train_time, accuracy_top_k

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 10
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n')

    train_step(model=model,
               data_loader=MNIST_train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_top_k,
               device=device)
    
    test_step(model=model,
               data_loader=MNIST_test_dataloader,
               loss_fn=loss_fn,
               accuracy_fn=accuracy_top_k,
               device=device)

# Print time taken
train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(train_time_start_on_cpu, train_time_end_on_cpu, str(next(model.parameters()).device))

### Training loop - For testing CPU and GPU (do not change)

In [ ]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
from model.evaluation import train_step_label, test_step_label # use torchmetrics.Accuracy()
from model.utilis import print_train_time, accuracy_fn

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n')

    train_step_label(model=model,
               data_loader=MNIST_train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_fn,
               device=device)
    
    test_step_label(model=model,
               data_loader=MNIST_test_dataloader,
               loss_fn=loss_fn,
               accuracy_fn=accuracy_fn,
               device=device)

# Print time taken
train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(train_time_start_on_cpu, train_time_end_on_cpu, str(next(model.parameters()).device))

### Calculate model results on test dataset

In [ ]:
model_results = test_step(model=model,
                           data_loader=MNIST_test_dataloader,
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_top_k,
                           device=device)

model_results